In [1]:
import sqlite3

In [4]:
con = sqlite3.connect(":memory:")
cur = con.execute("CREATE TABLE lang(name, first_appeared)")

# This is the named style used with executemany():
data = (
    {"name": "C", "year": 1972},
    {"name": "Fortran", "year": 1957},
    {"name": "Python", "year": 1991},
    {"name": "Go", "year": 2009},
)
cur.executemany("INSERT INTO lang VALUES(:name, :year)", data)
cur.execute("CREATE INDEX year ON lang(first_appeared)")
# This is the qmark style used in a SELECT query:
params = (1972,)
cur.execute("SELECT * FROM lang WHERE first_appeared = ?", params)
print(cur.fetchall())

[('C', 1972)]


```
SELECT a.DocID
  FROM Index a,Index b,Index c  
 WHERE a.Term=‘vehicle‘ AND
       b.Term=‘sales‘ AND
       c.Term=‘italy‘ AND
       a.DocID=b.DocID AND
       a.DocID=c.DocID;

DELETE FROM Query;
INSERT INTO Query    VALUES(‘vehicle‘,1);
INSERT INTO Query    VALUES(‘sales‘,1);
INSERT INTO Query    VALUES(‘italy‘,1);

SELECT i.DocID
  FROM Index i, Query q
 WHERE i.Term=q.Term
 GROUP BY i.DocID
HAVING COUNT(i.Term)=
    (SELECT COUNT(*) FROM QUERY)


	DELETE FROM Query;
	INSERT INTO Query	    VALUES(‘vehicle‘,1);
	INSERT INTO Query     VALUES(‘sales‘,1);
	INSERT INTO Query	    VALUES(‘italy‘,1);


	SELECT i.DocID, SUM(q.tf * t.idf * i.tf * t.idf)
	  FROM Query q, Index i, Term t
	 WHERE q.Term=t.Term AND
	       i.Term=t.Term
	 GROUP BY i.DocID
	 ORDER BY 2 DESC;

```